In [ ]:
# Challenge 4 - 2025/05/31
# - Implement a complete RAG pipeline with a Stuff Documents chain.
# - You must implement the chain manually.
# - Give a ConversationBufferMemory to the chain.
# - Use this document to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# - Ask the following questions to the chain:
#   - Is Aaronson guilty?
#   - What message did he write in the table?
#   - Who is Julia?

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/1984_ch03.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {"context": retriver, "question": RunnablePassthrough(), "history": load_memory}
    | prompt
    | llm
)

memory = ConversationBufferMemory()


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"question": question},
        {"answer": result.content},
    )
    print(result)

In [2]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [3]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" in the dust on the table.'


In [4]:
invoke_chain("Who is Julia?")

content='Julia is a character in the text who is mentioned as someone Winston loves and cares for deeply.'


In [5]:
memory.load_memory_variables({})

{'history': 'Human: Is Aaronson guilty?\nAI: Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.\nHuman: What message did he write in the table?\nAI: He wrote "2+2=5" in the dust on the table.\nHuman: Who is Julia?\nAI: Julia is a character in the text who is mentioned as someone Winston loves and cares for deeply.'}